Must run this code first to import required libraries.

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv
from urllib.error import HTTPError
from urllib.error import URLError
import sys

Enter the URL of the video of Youtube

In [ ]:
url = input("Enter URL:")
driver = webdriver.Chrome()
driver.get(url)

It will autometically scroll the page to load more comments

In [ ]:
x = 50
time.sleep(10)
prev_h = 0
while True:
    height = driver.execute_script("""
    function getActualHeight() {
    return Math.max(
    Math.max(document.body.scrollHeight, document.documentElement.scrollHeight),
    Math.max(document.body.offsetHeight, document.documentElement.offsetHeight),
    Math.max(document.body.clientHeight, document.documentElement.clientHeight),
    );
    }
    return getActualHeight();
    """)
    if prev_h > height:
        time.sleep(5)
    driver.execute_script(f"window.scrollTo({prev_h},{prev_h+x})")
    prev_h += x
    if prev_h > height:
        break
soup = BeautifulSoup(driver.page_source,'lxml')
print(soup)

Testing Every Features:

In [ ]:
commentSection = soup.find('div',class_='style-scope ytd-item-section-renderer style-scope ytd-item-section-renderer')
commentAmount = commentSection.find('span',class_='style-scope yt-formatted-string').text.strip()
title = soup.find("div",class_="style-scope ytd-watch-metadata")
title = title.find("yt-formatted-string",class_="style-scope ytd-watch-metadata").text.strip()

print(f"Title: {title}\n")
print(f"Total number of comment: {commentAmount}\n")

In [ ]:
singleComment = soup.find_all("ytd-comment-thread-renderer",class_="style-scope ytd-item-section-renderer")
print(f"Total comment sections are : {len(singleComment)}\n")
# print(singleComment)

In [ ]:
name = singleComment[1].find('span',class_='style-scope ytd-comment-renderer style-scope ytd-comment-renderer')

Error Handling

In [ ]:
def getName(elem):
    try:
        name = elem.find('span',class_='style-scope ytd-comment-renderer style-scope ytd-comment-renderer').text.strip()
    except AttributeError:
        name = ''
    return name

def getTime(elem):
    try:
        timeAgo = elem.find(class_='yt-simple-endpoint style-scope yt-formatted-string').text.strip()
    except AttributeError:
        timeAgo = ''
    return timeAgo

def getLike(elem):
    try:
        Like = elem.find('span',class_="style-scope ytd-comment-action-buttons-renderer").text.strip()
    except AttributeError:
        Like = ''
    return Like

def getComment(elem):
    try:
        comment = elem.find('yt-formatted-string',class_='style-scope ytd-comment-renderer').text.strip()
    except AttributeError:
        comment = ''
    return comment


Storing all the data into lists

In [ ]:
nameList,timeAgoList, LikeList, CommentList = [],[],[],[]
for i in singleComment:
    name = getName(i)
    nameList.append(name)
    timeAgo = getTime(i)
    timeAgoList.append(timeAgo)
    comment = getComment(i)
    CommentList.append(comment)
    Like = getLike(i)
    LikeList.append(Like)

Storing all the data into a DataFrame

In [ ]:
dic = {"Name":nameList,"Time Ago":timeAgo,"Like":LikeList,"Comment":CommentList}
df = pd.DataFrame(dic)
df

DataFrame to CSV

In [ ]:
# fileName = input("Enter the Name of CSV File: ")
df.to_csv('set-file-name.csv')